In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
import plotly.express as px


In [ ]:
sales = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='出貨明細')
budget = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols=('A:F'))                      

In [ ]:
sales['BG'] = sales.apply(lambda x: 'RFBU2' if 'RFDP' in x['品名'] else x['BG'], axis=1)

In [ ]:
sales_filter = sales[sales['狀態'].str.contains('出')]

In [ ]:
budget['產品'] = budget.apply(lambda x: '天線' if 'RF' in x['客戶名稱'] else '元件', axis=1)

In [ ]:
budget['預估業績'] = budget['預估業績'].apply(lambda x: '0' if x == ' -   ' else x)
budget['預估業績'] = budget['預估業績'].astype('int')

In [ ]:
sales_filter['產品'] = sales_filter.apply(lambda x: '天線' if 'RF' in x['BG'] or 'RFBU2' in x['BG'] else '元件', axis=1)

In [ ]:
sales_filter = sales_filter.groupby(['Group', '負責業務', '預交年份', '預交月份', '產品'])[['本國幣別NTD']].sum().reset_index()

In [ ]:
sales_filter.insert(1, '類型', '實績')

In [ ]:
sales_filter = sales_filter.reindex(columns=['Group', '類型', '負責業務', '預交年份', '預交月份', '本國幣別NTD', '產品'])

In [ ]:
sales_filter.columns = budget.columns

In [ ]:
sales_filter['預估業績'] = sales_filter['預估業績'].astype('int')

In [ ]:
sales_budget = pd.concat([budget, sales_filter], axis=0)

In [ ]:
customer_dict = {'Bourns': 'Bourns', 'Bourns RF': 'Bourns', 'Eaton':'Eaton',
                'Laird':'Larid', 'LF':'LF', 'LF-OEM':'LF OEM', 'WE':'WE',
                '信昌':'信昌', '東莞弘電':'東莞弘電', '藝感':'藝感', '華科': '華新科技', 
                'Mt System(Component)': 'MT-SYSTEM', 'Teltonika(Component)':'TELTONIKA', 
                'Mt System(RF)': 'MT-SYSTEM', 'Teltonika(RF)':'TELTONIKA', 'Te(RF)':'TE', 
                'Credence(India RF)':'Credence', 'Millenium(India RF)':'Millenium', 
                'Nexty(India RF)': 'Nexty', 'OSE(India RF)': 'OSE', 'Others/One Time PO(India RF)': 'Others/One Time PO',
                'Credence':'Credence', 'Eteily':'Eteily', 'FIRST EN':'FIRST EN', 'Fides':'Fides',
                'G -PLAST':'G-PLAST', 'Globalte':'Globalte', 'JBJ':'JBJ', 'LF OEM':'LF OEM', 'M.S.':'M.S.',
                'MATE':'MATE', 'MT-SYSTEM':'MT-SYSTEM', 'Nexty':'Nexty', 'SSF': 'SSF', 
                'Sancraft': 'Sancraft', 'TELTONIKA': 'TELTONIKA', 'Univa Te':'Univa Te', 'WiSilica':'WiSilica', 
                'Zebra':'Zebra', '華新科技':'華新科技', 'TE ':'TE'}

sales_budget['客戶名稱'] = sales_budget['客戶名稱'].map(customer_dict)

In [ ]:
from datetime import datetime

cur_month = datetime.today().strftime('%B')

In [ ]:
y = sales_budget[sales_budget['Month'] == cur_month]
y.groupby(['年份', '類型', 'Month'])[['預估業績']].sum().reset_index()

In [ ]:
sales_amber = sales_budget.groupby(['客戶名稱','類型', '產品', '負責業務', '年份', 'Month'])['預估業績'].sum().unstack('Month')

In [ ]:
# mon_order = ['January', 'February', 'March', 
#             'April', 'May', 'June', 'July', 'August',
#              'September', 'October', 'November', 'December']

# sales_budget['Month'] =pd.Categorical(sales_budget['Month'], ordered=True, categories=mon_order)

In [ ]:
sales_amber.reindex(columns=['January', 'February', 'March', 
            'April', 'May', 'June', 'July', 'August',
             'September', 'October', 'November', 'December'])

In [ ]:
# sales_budget.groupby([''])

fig = px.histogram(sales_budget, x='Month', y='預估業績', color='負責業務', facet_col='年份', facet_row='類型',
                  labels={'預估業績':'總金額', 'Month':'月份'}, title='實績VS預算')
fig.show()